In [2]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2

cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'Phone',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In',
'GDPR Compliant',
'Website',
'Comments',
'Do you live in Bali?']

In [3]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\wedding.csv'
zoho_df = pd.read_csv(path, dtype='string')
# zoho_df = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

zoho_df['Mobile'] = 'empty'
# zoho_df['Phone'] = 'empty'
zoho_df

,Record Id,Created Time,First Name,Last Name,Email,Phone,Mobile,Wedding Date,Messages,Lead Brand,...,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Country,Contact type,Enquiry sent to
0,zcrm_3584690000378130330,2022-06-27 15:28:11,Sophie,Turki,sophieturki@yahoo.fr,33659130908,empty,empty,We would like a very intimate wedding because ...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,true,France,empty,events@karmaresorts.com
1,zcrm_3584690000380133001,2022-07-01 22:17:36,Verie,Maikelin,maikelinverie@gmail.com,+6285783816330,empty,empty,Hubungi saya buat info karma kandara,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,empty,events@karmaresorts.com
2,zcrm_3584690000380211111,2022-07-03 07:26:22,Delfina,Dimoski,del.dimoski@gmail.com,+61401155560,empty,empty,empty,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Australia,empty,events@karmaresorts.com
3,zcrm_3584690000380218350,2022-07-03 21:40:11,Syahrial,Muharam,syahrialm@hotmail.com,62818675273,empty,empty,info,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,empty,empty,events@karmaresorts.com
4,zcrm_3584690000380880118,2022-07-05 16:25:08,Merrysa,Gudat,merrysa.dirganova@gmail.com,+4917683176782,empty,empty,We would like to enquire for an intimate famil...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,true,Germany,empty,events@karmaresorts.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7023,zcrm_3584690000534590044,2025-03-26 06:35:01,Abi Richards,Abi Richards,abigailmrichards2001@gmail.com,+447946524878,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
7024,zcrm_3584690000534609033,2025-03-26 02:07:04,Sophie,Sophie,sophiesparrowsmith@yahoo.co.uk,+447864842615,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
7025,zcrm_3584690000534614033,2025-03-26 01:54:13,Sophie,Sophie,sophiesparrowsmoth@yahoo.co.uk,+447864842615,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
7026,zcrm_3584690000534656034,2025-03-26 07:10:40,Amy,Stofberg,amystoffies@gmail.com,+61400048869,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Australia,Email and Phone,events@karmaresorts.com


In [4]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [5]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

'Email'

In [6]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [7]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Record Id,Created Time,First Name,Last Name,Email,Phone,Mobile,Wedding Date,Messages,Lead Brand,...,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Country,Contact type,Enquiry sent to
73,zcrm_3584690000389261021,2022-08-09 11:52:28,Brittney,Massey,brittneymassey50@gmail.com,19122374993,empty,empty,"hi, we would like more information on booking ...",Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,United States,empty,events@karmaresorts.com
106,zcrm_3584690000391060062,2022-08-15 18:10:36,Kaitlyn,Kaitlyn,kaitlyncross79@gmail.com,447704928322,empty,empty,empty,Karma Resorts,...,Karma Salford Hall - Weddings at Karma Salford...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,empty,empty
148,zcrm_3584690000393317037,2022-08-20 17:46:02,Megan,Walters,meganewalters@outlook.com,447944826524,empty,empty,"Hi, looking for a Saturday in March 2023 for a...",Karma Resorts,...,Karma Salford Hall - Weddings at Karma Salford...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,empty,empty
257,zcrm_3584690000398105101,2022-09-11 17:58:13,Sophie,Wadey,sophiewadey@gmail.com,447572011975,empty,empty,Enquiry for a small wedding 30 day guests and ...,Karma Resorts,...,Karma Salford Hall - Weddings at Karma Salford...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,empty,empty
355,zcrm_3584690000402730184,2022-10-03 06:02:41,Ruth,Pauline,ruthpaulinehutapea@gmail.com,642108815121,empty,empty,Looking at holding a wedding event 80 - 100 pa...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,New Zealand,empty,events@karmaresorts.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6984,zcrm_3584690000533630097,2025-03-23 23:47:13,empty,Alexandria Parry,alexandriaparry@hotmail.com,447532691220,empty,empty,empty,Karma Resorts,...,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,United Kingdom,Email and Phone,events@karmalakeofmenteith.com
6986,zcrm_3584690000533676315,2025-03-23 20:36:45,empty,Halina,halinafried@hotmail.com,17543681336,empty,empty,empty,Karma Resorts,...,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,false,United States,Email and Phone,events@karmalakeofmenteith.com
6987,zcrm_3584690000533676379,2025-03-24 05:29:10,empty,Carol,carola18@live.co.uk,447863992359,empty,empty,empty,Karma Resorts,...,Karma Lake Of Menteith - Download Wedding Broc...,Web Enquiry,Karma Lake Of Menteith,https://karmagroup.com/weddings-at-karma-lake-...,true,true,true,United Kingdom,Email and Phone,events@karmalakeofmenteith.com
6988,zcrm_3584690000533706217,2025-03-23 20:11:15,Mellissa,Hunter,mellissahunter89@gmail.com,61431124114,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Australia,Email and Phone,events@karmaresorts.com


In [8]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,First Name,Last Name,Email,Phone,Mobile,Wedding Date,Messages,Lead Brand,...,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Country,Contact type,Enquiry sent to
0,zcrm_3584690000378130330,2022-06-27 15:28:11,Sophie,Turki,sophieturki@yahoo.fr,33659130908,empty,empty,We would like a very intimate wedding because ...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,true,France,empty,events@karmaresorts.com
1,zcrm_3584690000380133001,2022-07-01 22:17:36,Verie,Maikelin,maikelinverie@gmail.com,6285783816330,empty,empty,Hubungi saya buat info karma kandara,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,empty,events@karmaresorts.com
2,zcrm_3584690000380211111,2022-07-03 07:26:22,Delfina,Dimoski,del.dimoski@gmail.com,61401155560,empty,empty,empty,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Australia,empty,events@karmaresorts.com
3,zcrm_3584690000380218350,2022-07-03 21:40:11,Syahrial,Muharam,syahrialm@hotmail.com,62818675273,empty,empty,info,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,empty,empty,events@karmaresorts.com
4,zcrm_3584690000380880118,2022-07-05 16:25:08,Merrysa,Gudat,merrysa.dirganova@gmail.com,4917683176782,empty,empty,We would like to enquire for an intimate famil...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,true,Germany,empty,events@karmaresorts.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6890,zcrm_3584690000534590044,2025-03-26 06:35:01,Abi Richards,Abi Richards,abigailmrichards2001@gmail.com,447946524878,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
6891,zcrm_3584690000534609033,2025-03-26 02:07:04,Sophie,Sophie,sophiesparrowsmith@yahoo.co.uk,447864842615,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
6892,zcrm_3584690000534614033,2025-03-26 01:54:13,Sophie,Sophie,sophiesparrowsmoth@yahoo.co.uk,447864842615,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
6893,zcrm_3584690000534656034,2025-03-26 07:10:40,Amy,Stofberg,amystoffies@gmail.com,61400048869,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Australia,Email and Phone,events@karmaresorts.com


In [9]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Record Id,Created Time,First Name,Last Name,Email,Phone,Mobile,Wedding Date,Messages,Lead Brand,...,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Country,Contact type,Enquiry sent to
5346,zcrm_3584690000505442145,2024-08-22 02:02:06,empty,Test,morveee@aol.com,447777777777,empty,empty,empty,Karma Resorts,...,Karma Lake Of Menteith - Download Wedding Broc...,Web Enquiry,Karma Lake Of Menteith,https://karmagroup.com/weddings-at-karma-lake-...,true,true,true,United Kingdom,Email and Phone,events@karmalakeofmenteith.com
5986,zcrm_3584690000515651195,2024-11-19 15:26:38,shiju test,shaji test,memiko1618@merotx.com,916362457127,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,India,Email and Phone,events@karmaresorts.com
6466,zcrm_3584690000522472118,2025-01-27 15:44:19,TEST,TEST,test@gmail.com,919088764211,empty,empty,empty,Karma Resorts,...,Karma Lake of Menteith- Weddings at Karma Lake...,Web Enquiry,Karma Lake of Menteith,https://karmagroup.com/weddings-at-karma-lake-...,true,true,false,India,Email and Phone,empty
6519,zcrm_3584690000523252889,2025-02-03 18:42:45,TEST,TEST,test@gmail.com,919877665654,empty,empty,empty,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,India,Email and Phone,events@karmaresorts.com
6525,zcrm_3584690000523362049,2025-02-03 18:51:14,TEST,TEST,test@gmail.com,918876543112,empty,empty,empty,Karma Resorts,...,Karma Lake of Menteith- Weddings at Karma Lake...,Web Enquiry,Karma Lake of Menteith,https://karmagroup.com/weddings-at-karma-lake-...,true,true,false,India,Email and Phone,empty
6526,zcrm_3584690000523366078,2025-02-03 18:44:09,Rajesh,Karma,test@gmail.com,919776543122,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,India,Email and Phone,events@karmaresorts.com
6545,zcrm_3584690000523640077,2025-02-05 13:16:45,TEST,TEST,test@gmail.com,918876543121,empty,empty,empty,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,India,Email and Phone,events@karmaresorts.com
6658,zcrm_3584690000525220044,2025-02-20 19:21:07,TEST,TEST,rajesh.kumar@karmagroup.com,919877666554,empty,empty,"Please ignore this ,as it is test",Karma Resorts,...,Karma Salford Hall - Weddings at Karma Salford...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,false,India,Email and Phone,empty
6825,zcrm_3584690000531086001,2025-03-17 16:13:48,TEST,TEST,rajesh.kumar@karmagroup.com,919876655432,empty,empty,please ignore this is a test mail,Karma Resorts,...,Karma Salford Hall - Weddings at Karma Salford...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,false,India,Email and Phone,empty


In [10]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [11]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [12]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [13]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Record Id,Created Time,First Name,Last Name,Email,Phone,Mobile,Wedding Date,Messages,Lead Brand,...,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Country,Contact type,Enquiry sent to


In [14]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,First Name,Last Name,Email,Phone,Mobile,Wedding Date,Messages,Lead Brand,...,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Country,Contact type,Enquiry sent to
0,zcrm_3584690000378130330,2022-06-27 15:28:11,Sophie,Turki,sophieturki@yahoo.fr,33659130908,empty,empty,We would like a very intimate wedding because ...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,true,France,empty,events@karmaresorts.com
1,zcrm_3584690000380133001,2022-07-01 22:17:36,Verie,Maikelin,maikelinverie@gmail.com,6285783816330,empty,empty,Hubungi saya buat info karma kandara,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,empty,events@karmaresorts.com
2,zcrm_3584690000380211111,2022-07-03 07:26:22,Delfina,Dimoski,del.dimoski@gmail.com,61401155560,empty,empty,empty,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Australia,empty,events@karmaresorts.com
3,zcrm_3584690000380218350,2022-07-03 21:40:11,Syahrial,Muharam,syahrialm@hotmail.com,62818675273,empty,empty,info,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,empty,empty,events@karmaresorts.com
4,zcrm_3584690000380880118,2022-07-05 16:25:08,Merrysa,Gudat,merrysa.dirganova@gmail.com,4917683176782,empty,empty,We would like to enquire for an intimate famil...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,true,Germany,empty,events@karmaresorts.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6881,zcrm_3584690000534590044,2025-03-26 06:35:01,Abi Richards,Abi Richards,abigailmrichards2001@gmail.com,447946524878,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
6882,zcrm_3584690000534609033,2025-03-26 02:07:04,Sophie,Sophie,sophiesparrowsmith@yahoo.co.uk,447864842615,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
6883,zcrm_3584690000534614033,2025-03-26 01:54:13,Sophie,Sophie,sophiesparrowsmoth@yahoo.co.uk,447864842615,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
6884,zcrm_3584690000534656034,2025-03-26 07:10:40,Amy,Stofberg,amystoffies@gmail.com,61400048869,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Australia,Email and Phone,events@karmaresorts.com


In [15]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,First Name,Last Name,Email,Phone,Mobile,Wedding Date,Messages,Lead Brand,...,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Country,Contact type,Enquiry sent to
450,zcrm_3584690000405820115,2022-10-21 20:23:22,Miss,Elise Williams,events@karmasalfordhall.com,441386871300,empty,empty,empty,Karma Resorts,...,Karma Salford Hall - Weddings at Karma Salford...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,empty,empty
563,zcrm_3584690000409569251,2022-11-09 18:04:59,[First-Name],[Last-Name],events@karmasalfordhall.com,447552069179,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,empty,empty
682,zcrm_3584690000415832001,2022-12-08 21:36:33,Shaun,Duenhofen,shaun.duenhofen@karmakandara.com,6281138006892,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,empty,events@karmaresorts.com
1053,zcrm_3584690000429061041,2023-02-14 20:41:52,Elise,Williams,reception@karmasalfordhall.com,447764618513,empty,empty,empty,Karma Resorts,...,Karma Lake Of Menteith - Download Wedding Broc...,Web Enquiry,Karma Lake Of Menteith,https://karmagroup.com/weddings-at-karma-lake-...,true,true,true,United Kingdom,empty,events@karmalakeofmenteith.com
3241,zcrm_3584690000470644676,2023-11-22 11:57:12,Lita,Rahayu,novalita.rahayu@karmagroup.com,6282237453320,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,Email and Phone,events@karmaresorts.com
3766,zcrm_3584690000484131001,2024-02-16 23:53:58,empty,Adrian Willmers,gm@karmasalfordhall.com,447827542105,empty,empty,empty,Karma Resorts,...,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,true,United Kingdom,Email and Phone,events@karmalakeofmenteith.com
3848,zcrm_3584690000485596088,2024-02-29 15:44:05,empty,Shivani,shivani.yeshwant@karmagroup.com,917682075005,empty,empty,empty,Karma Resorts,...,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,false,India,Email and Phone,events@karmalakeofmenteith.com
4125,zcrm_3584690000491983036,2024-04-16 18:26:51,Stephen,Halim,salescoordinator@karmaresorts.com,6281617722971,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,Email and Phone,events@karmaresorts.com
4133,zcrm_3584690000492392060,2024-04-18 15:56:41,Tanya,Klimentyeva,tanya.klimentyeva@karmakandara.com,6281138305940,empty,empty,empty,Karma Resorts,...,Karma Kandara -Download Wedding Brochure at Ka...,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,Email and Phone,events@karmaresorts.com
4342,zcrm_3584690000496311706,2024-05-21 12:46:21,Maida,Maida,maida.pratiwi@karmaresorts.com,62878900098978,empty,empty,empty,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,Email and Phone,events@karmaresorts.com


In [16]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,First Name,Last Name,Email,Phone,Mobile,Wedding Date,Messages,Lead Brand,...,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Country,Contact type,Enquiry sent to
0,zcrm_3584690000378130330,2022-06-27 15:28:11,Sophie,Turki,sophieturki@yahoo.fr,33659130908,empty,empty,We would like a very intimate wedding because ...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,true,France,empty,events@karmaresorts.com
1,zcrm_3584690000380133001,2022-07-01 22:17:36,Verie,Maikelin,maikelinverie@gmail.com,6285783816330,empty,empty,Hubungi saya buat info karma kandara,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Indonesia,empty,events@karmaresorts.com
2,zcrm_3584690000380211111,2022-07-03 07:26:22,Delfina,Dimoski,del.dimoski@gmail.com,61401155560,empty,empty,empty,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,Australia,empty,events@karmaresorts.com
3,zcrm_3584690000380218350,2022-07-03 21:40:11,Syahrial,Muharam,syahrialm@hotmail.com,62818675273,empty,empty,info,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,false,empty,empty,events@karmaresorts.com
4,zcrm_3584690000380880118,2022-07-05 16:25:08,Merrysa,Gudat,merrysa.dirganova@gmail.com,4917683176782,empty,empty,We would like to enquire for an intimate famil...,Karma Resorts,...,Karma Kandara - Weddings at Karma Kandara,Web Enquiry,Karma Kandara,https://karmagroup.com/weddings-at-karma-kanda...,true,true,true,Germany,empty,events@karmaresorts.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,zcrm_3584690000534581063,2025-03-25 20:32:49,empty,Daniel Rosenberg,daniellouisrosenberg@gmail.com,16098195113,empty,empty,empty,Karma Resorts,...,Karma La Herriza - Download Wedding Brochure a...,Web Enquiry,Karma La Herriza,https://karmagroup.com/weddings-at-karma-la-he...,true,true,false,United States,Email and Phone,events@karmalakeofmenteith.com
6860,zcrm_3584690000534590044,2025-03-26 06:35:01,Abi Richards,Abi Richards,abigailmrichards2001@gmail.com,447946524878,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
6861,zcrm_3584690000534609033,2025-03-26 02:07:04,Sophie,Sophie,sophiesparrowsmith@yahoo.co.uk,447864842615,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty
6862,zcrm_3584690000534614033,2025-03-26 01:54:13,Sophie,Sophie,sophiesparrowsmoth@yahoo.co.uk,447864842615,empty,empty,empty,Karma Resorts,...,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,true,true,true,United Kingdom,Email and Phone,empty


In [17]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [18]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [19]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [20]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

164

In [21]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 7028
Jumlah Format Email Salah : 0
Jumlah Setelah Clean Format Email Salah : 7028
Jumlah Duplicate (Phone, Mobile dan Email) : 133
Jumlah Setelah Clean Duplicate : 6895
Jumlah Lead Test : 9
Jumlah Setelah Clean Lead Test : 6886
Jumlah Email OTA : 0
Jumlah Setelah Clean Email OTA : 6886
Jumlah Email Karma : 22
Jumlah Setelah Clean Email Karma : 6864
Jumlah Wash Member : 140
Jumlah Wash Booking : 2
Jumlah Setelah Wash Member : 6722
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 0
Jumlah Setelah Wash Phone Email Blank : 6722
Contact Type
Jumlah Email Only : 33
Jumlah Phone Only : 0
Jumlah Email dan Phone : 6689


In [22]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 7028
Jumlah Wash Dedup : 164
Jumlah Wash Member : 140
Jumlah Wash Booking : 2
Jumlah Data Akhir : 6722


In [23]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

# zoho_df.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\clean purple.csv', index=False)
zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\wedding-clean.xlsx', index=False)